In [64]:
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
import os
from azure.identity import AzureCliCredential
from azure.mgmt.resource import SubscriptionClient

In [65]:
credential = AzureCliCredential()
subscription_client = SubscriptionClient(credential)

subscription_id = next(subscription_client.subscriptions.list())
subs_id = subscription_id.subscription_id
print(subs_id)

542414c9-86c5-4daf-9ef7-a109a8241211


In [66]:
# Step 1: Provision a resource group
resource_client = ResourceManagementClient(credential, subs_id)

In [67]:
RESOURCE_GROUP_NAME = "PythonAzureExample-VM-rg"
LOCATION = "westus2"

In [68]:
rg_result = resource_client.resource_groups.create_or_update(RESOURCE_GROUP_NAME,
    {
        "location": LOCATION
    }
)

In [69]:
print(f"Provisioned resource group {rg_result.name} in the {rg_result.location} region")

Provisioned resource group PythonAzureExample-VM-rg in the westus2 region


In [70]:
# Step 2: provision a virtual network
VNET_NAME = "python-example-vnet"
SUBNET_NAME = "python-example-subnet"
IP_NAME = "python-example-ip"
IP_CONFIG_NAME = "python-example-ip-config"
NIC_NAME = "python-example-nic"

In [71]:
network_client = NetworkManagementClient(credential, subs_id)

# Provision the virtual network and wait for completion
poller = network_client.virtual_networks.begin_create_or_update(RESOURCE_GROUP_NAME,
    VNET_NAME,
    {
        "location": LOCATION,
        "address_space": {
            "address_prefixes": ["10.0.0.0/16"]
        }
    }
)

In [72]:
vnet_result = poller.result()

In [73]:
print(f"Provisioned virtual network {vnet_result.name} with address prefixes {vnet_result.address_space.address_prefixes}") 
print(vnet_result)

Provisioned virtual network python-example-vnet with address prefixes ['10.0.0.0/16']
{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/virtualNetworks/python-example-vnet', 'name': 'python-example-vnet', 'type': 'Microsoft.Network/virtualNetworks', 'location': 'westus2', 'tags': None, 'extended_location': None, 'etag': 'W/"8a99ffea-474f-4205-a006-cf28adc5434c"', 'address_space': <azure.mgmt.network.v2021_02_01.models._models_py3.AddressSpace object at 0x7f87a095a430>, 'dhcp_options': None, 'flow_timeout_in_minutes': None, 'subnets': [], 'virtual_network_peerings': [], 'resource_guid': 'c3362302-7bee-4668-99de-9d4449b6ac94', 'provisioning_state': 'Succeeded', 'enable_ddos_protection': False, 'enable_vm_protection': None, 'ddos_protection_plan': None, 'bgp_communities': None, 'ip_allocations': None}


In [74]:
from azure.mgmt.network.v2020_06_01.models import NetworkSecurityGroup
nsg_name = "testnsg"

nsg_params = NetworkSecurityGroup(id= "testnsg", location=LOCATION, tags={ "name" : "testnsg"})  

poller = network_client.network_security_groups.begin_create_or_update(RESOURCE_GROUP_NAME, "testnsg", parameters=nsg_params) 
nsg_result = poller.result()

print(f"{nsg_result}")

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/networkSecurityGroups/testnsg', 'name': 'testnsg', 'type': 'Microsoft.Network/networkSecurityGroups', 'location': 'westus2', 'tags': {'name': 'testnsg'}, 'etag': 'W/"014860d4-5540-412c-b94e-000d2d21b7f3"', 'security_rules': [], 'default_security_rules': [<azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f8790303a00>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f8790303ee0>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87903031f0>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f8790303c40>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87801c2970>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87801c2f10>], 'network_interfaces': None, 'subnets': No

In [75]:
from azure.mgmt.network.v2020_06_01.models import SecurityRule
sec_rule_name = "testrule"
sec_rule_params = SecurityRule(
    id="testrule", 
    name=sec_rule_name, 
    protocol="Tcp", 
    source_port_range="*",
    destination_port_range="22",
    source_address_prefix="*",
    destination_address_prefix="*",
    access="Allow",
    direction="inbound", 
    priority=100,
)

poller = network_client.security_rules.begin_create_or_update(RESOURCE_GROUP_NAME,nsg_name,security_rule_name="testrule",security_rule_parameters=sec_rule_params) 
sec_rule_result = poller.result()
print(f"{sec_rule_result}")

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/networkSecurityGroups/testnsg/securityRules/testrule', 'name': 'testrule', 'etag': 'W/"1212e398-6034-44a5-aec1-0bfc08b37283"', 'type': 'Microsoft.Network/networkSecurityGroups/securityRules', 'description': None, 'protocol': 'Tcp', 'source_port_range': '*', 'destination_port_range': '22', 'source_address_prefix': '*', 'source_address_prefixes': [], 'source_application_security_groups': None, 'destination_address_prefix': '*', 'destination_address_prefixes': [], 'destination_application_security_groups': None, 'source_port_ranges': [], 'destination_port_ranges': [], 'access': 'Allow', 'priority': 100, 'direction': 'Inbound', 'provisioning_state': 'Succeeded'}


In [76]:
# step 3: Provision the subnet and wait for completion
poller = network_client.subnets.begin_create_or_update(RESOURCE_GROUP_NAME, 
    VNET_NAME, SUBNET_NAME,                                                 
    { "address_prefix": "10.0.0.0/24", 
     "network_security_group":{
        "id": nsg_result.id,
    } },
)
subnet_result = poller.result()

print(f"Provisioned virtual subnet {subnet_result.name} with address prefix {subnet_result.address_prefix}")


Provisioned virtual subnet python-example-subnet with address prefix 10.0.0.0/24


In [77]:
print(subnet_result)

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/virtualNetworks/python-example-vnet/subnets/python-example-subnet', 'name': 'python-example-subnet', 'etag': 'W/"6d17eff2-1ec5-4f14-83e1-2be637feec12"', 'type': 'Microsoft.Network/virtualNetworks/subnets', 'address_prefix': '10.0.0.0/24', 'address_prefixes': None, 'network_security_group': <azure.mgmt.network.v2021_02_01.models._models_py3.NetworkSecurityGroup object at 0x7f87a07a1490>, 'route_table': None, 'nat_gateway': None, 'service_endpoints': None, 'service_endpoint_policies': None, 'private_endpoints': None, 'ip_configurations': None, 'ip_configuration_profiles': None, 'ip_allocations': None, 'resource_navigation_links': None, 'service_association_links': None, 'delegations': [], 'purpose': None, 'provisioning_state': 'Succeeded', 'private_endpoint_network_policies': 'Enabled', 'private_link_service_network_policies': 'Enabl

In [78]:
# Step 4: Provision an IP address and wait for completion
poller = network_client.public_ip_addresses.begin_create_or_update(RESOURCE_GROUP_NAME,
    IP_NAME,
    {
        "location": LOCATION,
        "sku": { "name": "Standard" },
        "public_ip_allocation_method": "Static",
        "public_ip_address_version" : "IPV4"
    }
)

In [79]:
ip_address_result = poller.result()
print(f"Provisioned public IP address {ip_address_result.name} with address {ip_address_result.ip_address}")

Provisioned public IP address python-example-ip with address 52.229.11.108


In [80]:
# Step 5: Provision the network interface client
poller = network_client.network_interfaces.begin_create_or_update(RESOURCE_GROUP_NAME,
    NIC_NAME, 
    {
        "location": LOCATION,
        "ip_configurations": [ {
            "name": IP_CONFIG_NAME,
            "subnet": { "id": subnet_result.id },
            "public_ip_address": {"id": ip_address_result.id }
        }]
    }
)

In [81]:
nic_result = poller.result()
print(f"Provisioned network interface client {nic_result.name}")

Provisioned network interface client python-example-nic


In [82]:
# Step 6: Provision the virtual machine
compute_client = ComputeManagementClient(credential, subs_id)

VM_NAME = "ExampleVM"
USERNAME = "azureuser"
PASSWORD = "ChangePa$$w0rd24"

In [83]:
poller = compute_client.virtual_machines.begin_create_or_update(RESOURCE_GROUP_NAME, VM_NAME,
                                                               {
                                                                   "location": LOCATION,
                                                                   "storage_profile":{
                                                                       "image_reference":{
                                                                           "publisher":'Canonical',
                                                                           "offer":"UbuntuServer",
                                                                           "sku":"16.04.0-LTS",
                                                                           "version":"latest"
                                                                       }
                                                                   },
                                                                   "hardware_profile":{
                                                                       "vm_size":"Standard_DS1_v2"
                                                                   },
                                                                   "os_profile":{
                                                                       "computer_name":VM_NAME,
                                                                       "admin_username":USERNAME,
                                                                       "admin_password":PASSWORD
                                                                   },
                                                                   "network_profile":{
                                                                       "network_interfaces":[{
                                                                           "id":nic_result.id,
                                                                       }]
                                                                   },
                                                               })

In [84]:
vm_result = poller.result()

In [85]:
print(f"Provisioned virtual machine{vm_result.name}")

Provisioned virtual machineExampleVM
